In [ ]:
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
import pdfo
import scipy.optimize as opt

In [ ]:
data = np.load("data/dataset/sel_voxels_from_4_ds.npy")
print(data.shape)

In [ ]:
cov_m = np.cov(data.T)

sn.heatmap(cov_m)
plt.show()

In [ ]:
print(np.min(cov_m))

In [ ]:
e_val, e_vec = np.linalg.eig(cov_m)

In [ ]:
plt.plot(e_val, marker='o')
plt.yscale("log")
plt.show()
plt.yscale("log")
plt.plot(e_val[:10], marker='o')
plt.show()

In [ ]:
cen = np.array([np.average(x) for x in data.T])

ti_seq = np.array([*list(range(50, 400, 25)),
          *list(range(400, 1000, 10)),
          1000, 1030, 1050, 1080, 1100, 1130, 1150, 1180, 1200, 1230, 1250, 1280, 1300, 1330, 1350, 1380,
          1400, 1450, 1500, 1550, 1600, 1650, 1700,
          1800, 1900, 2000, 2100, 2200, 2300, 2500, 3000])

plt.figure(figsize=(15, 10))
for i in range(6):
    plt.plot(ti_seq, e_vec[:,i], label="{}".format(i))
plt.legend()
plt.show()

In [ ]:
plt.plot(ti_seq, cen)
plt.show()

In [ ]:
test_data = data[79]
decentr = test_data - cen

vec_count = 7
coefs = []
for i in range(105):
    l = np.linalg.norm(e_vec[:,i])
    coefs.append(sum(decentr * e_vec[:,i]) / (l ** 2))

rec = np.copy(cen)
for i in range(vec_count):
    rec += coefs[i] * e_vec[:,i]

s = np.copy(cen)
for i in range(6):
    s += coefs[i] * e_vec[:,i]

plt.plot(ti_seq, test_data, label="data")
plt.plot(ti_seq, rec, label="7")
plt.plot(ti_seq, s, label="6")
plt.legend()
plt.show()

In [ ]:
def model(TI,T1,M0=1.):
    return M0*np.abs(1.-2.*np.exp(-TI/T1))

def modcurve(T1,M0=1.):
    return model(ti_seq,T1,M0)

def curvecoord(curve,base):
    coord = np.dot(base.T,curve)
    bnorm = np.linalg.norm(base,axis=0)
    coord /= bnorm**2
    return coord

def curveproj(curve,base):
    coord = curvecoord(curve,base)
    proj = np.sum(base * coord,axis=1) 
    return coord,proj

def curveres(curve,base):
    _,proj = curveproj(curve,base)
    return np.linalg.norm(curve-proj)

In [ ]:
# kolik rozmeru podprostoru nas zajima

components = 7
base = e_vec[:,:components]

In [ ]:
# Pro kazde T1 z Trange a nekolik zafixovanych hodnot M0 generujeme krivky
# kazda se posune do stredu, udela jeji projekce do vybraneho pdprostoru,
# zmeri se vzdalenost krivky od projekce

T1range = range(400,1800,5)
M0range = [100,200,300,500,800,1000]

plt.figure(figsize=(12,8))
for M0 in M0range:
    res = []
    for T1 in T1range:
        t1curve=modcurve(T1,M0)
        t1cen = t1curve - cen
        res.append(curveres(t1cen,base))
    
    plt.plot(T1range,res,label=str(M0))
    
plt.legend()
plt.show()

In [ ]:
# T1start = np.array([660,716,800,950,1180,1620])
T1start = np.array([580,650,710,810,1000,1250,1580])

models = []
#for t1 in range(500,1900,200):
for t1 in T1start:
    models.append(modcurve(t1,1200))
    
models = np.array(models).T
avgm = np.average(models,axis=1)

plt.plot(ti_seq,avgm)
plt.show()

In [ ]:
# Alternativne M0 spocitane tak, ze se stred nekolika krivek posune (hledanim spravneho M0) 
# co nejbliz stredu dat

M0 = 1200 * curvecoord(cen,avgm)
print(M0)

plt.figure(figsize=(12,8))
plt.plot(T1range, list(map(lambda T1: curveres(modcurve(T1,M0),base),T1range)))
plt.show()

In [ ]:
# A nejdivoceji ... pro kazde T1 optimalizujeme zvlast M0 tak, aby vzdalenost od poprostoru
# byla co nejmensi

plt.figure(figsize=(12,8))

T1range = list(range(400,3000,10))

mins = [
    opt.brent(lambda M0: curveres(modcurve(T1,M0)-cen,base),brack=(0,1000))
    for T1 in T1range
]

plt.plot(T1range,list(map(lambda TM: curveres(modcurve(*TM)-cen,base),zip(T1range,mins))),label='mindist')
plt.plot(T1range,np.array(mins) / 5,label='M0 / 5')
plt.legend()
plt.show()